 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
!ngrok authtoken 2xuatcbmHbDx33buojBGNGUkDGn_4ovYEiwmVDLXDwBC4eFFc

/bin/bash: line 1: ngrok: command not found


new ways


now webpage side page

In [ ]:
SPOONACULAR_API_KEY = "33323d39b2d54c42ba326fc2ec35cf32"

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import requests
import json
st.set_page_config(page_title="Python Recipe Finder", layout="centered")

st.title("🍜 Python Recipe Finder")

# --- Configuration ---
# Get API key from Streamlit secrets
# Ensure you have a .streamlit/secrets.toml file in your app's root directory
# containing: SPOONACULAR_API_KEY = "YOUR_ACTUAL_SPOONACULAR_API_KEY_HERE"
API_KEY = st.secrets["SPOONACULAR_API_KEY"] # This is the correct way to access it
BASE_URL = "https://api.spoonacular.com/"

# --- Spoonacular API Functions ---

def search_recipes(query, number=10):
    """Searches for recipes by name."""
    endpoint = "recipes/complexSearch"
    params = {
        "query": query,
        "number": number,
        "apiKey": API_KEY
    }
    try:
        response = requests.get(f"{BASE_URL}{endpoint}", params=params)
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json().get("results", [])
    except requests.exceptions.RequestException as e:
        st.error(f"Error searching for recipes: {e}")
        return []

def get_recipe_details(recipe_id):
    """Fetches detailed information for a specific recipe."""
    endpoint = f"recipes/{recipe_id}/information"
    params = {
        "includeNutrition": True, # Get nutrition data for calories
        "apiKey": API_KEY
    }
    try:
        response = requests.get(f"{BASE_URL}{endpoint}", params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        st.error(f"Error fetching recipe details: {e}")
        return None

# --- Streamlit UI Layout ---

# Input for recipe name
recipe_name_input = st.text_input(
    "Enter a recipe name (e.g., pasta, chicken soup)",
    key="recipe_name_input"
)

# Search button
if st.button("Search Recipes"):
    if recipe_name_input:
        st.session_state.search_query = recipe_name_input # Store query in session state
        st.session_state.current_page = "search_results" # Navigate to search results
        st.rerun() # Rerun the app to show results
    else:
        st.warning("Please enter a recipe name to search.")

# --- Navigation and Display Logic ---

# Initialize session state for navigation if not already set
if 'current_page' not in st.session_state:
    st.session_state.current_page = "search_input" # Default page
if 'search_query' not in st.session_state:
    st.session_state.search_query = ""
if 'selected_recipe_id' not in st.session_state:
    st.session_state.selected_recipe_id = None


# Display logic based on current_page in session state
if st.session_state.current_page == "search_results":
    if st.session_state.search_query:
        st.subheader(f"Search Results for '{st.session_state.search_query}'")
        recipes = search_recipes(st.session_state.search_query)

        if recipes:
            for recipe in recipes:
                col1, col2 = st.columns([3, 1])
                with col1:
                    st.write(f"**{recipe['title']}**")
                with col2:
                    # Store selected recipe ID in session state and change page
                    if st.button("View Details", key=f"view_{recipe['id']}"):
                        st.session_state.selected_recipe_id = recipe['id']
                        st.session_state.current_page = "recipe_details"
                        st.rerun() # Rerun to show details
            st.markdown("---") # Separator
            if st.button("Back to Search Input", key="back_to_input_from_results"):
                st.session_state.current_page = "search_input"
                st.session_state.search_query = ""
                st.rerun()
        else:
            st.info("No recipes found for your search. Try a different name.")
            if st.button("Back to Search Input", key="back_to_input_no_results"):
                st.session_state.current_page = "search_input"
                st.session_state.search_query = ""
                st.rerun()
    else: # If search_query is empty, go back to input state
        st.session_state.current_page = "search_input"
        st.rerun()

elif st.session_state.current_page == "recipe_details":
    if st.session_state.selected_recipe_id:
        st.subheader("Recipe Details")
        details = get_recipe_details(st.session_state.selected_recipe_id)

        if details:
            st.markdown(f"## {details.get('title', 'N/A')}")
            st.write(f"**Preparation Time:** {details.get('readyInMinutes', 'N/A')} minutes")
            st.write(f"**Servings:** {details.get('servings', 'N/A')}")

            # Extract Calories
            calories = "N/A"
            if details.get('nutrition') and details['nutrition'].get('nutrients'):
                for nutrient in details['nutrition']['nutrients']:
                    if nutrient.get('title') == 'Calories':
                        calories = f"{nutrient.get('amount', 'N/A')} {nutrient.get('unit', '')}"
                        break
            st.write(f"**Calories:** {calories}")

            if details.get('sourceUrl'):
                st.markdown(f"**Source:** [Link to Recipe]({details['sourceUrl']})")

            st.markdown("### Ingredients:")
            if details.get('extendedIngredients'):
                ingredients_list = "\n".join([f"- {ing['original']}" for ing in details['extendedIngredients']])
                st.markdown(ingredients_list)
            else:
                st.info("No detailed ingredients available.")

            st.markdown("### Instructions:")
            if details.get('analyzedInstructions'):
                # Handle structured steps
                for instruction_set in details['analyzedInstructions']:
                    steps_md = ""
                    for step in instruction_set.get('steps', []):
                        steps_md += f"{step.get('number')}. {step.get('step')}\n"
                    st.markdown(steps_md)
            elif details.get('instructions'):
                # Handle single string instructions (might be HTML)
                st.markdown(details['instructions'], unsafe_allow_html=True) # Use unsafe_allow_html if instructions can contain HTML
            else:
                st.info("No detailed instructions available.")
        else:
            st.warning("Could not load recipe details.")

        # Back button
        if st.button("Back to Search Results", key="back_to_results_from_details"):
            st.session_state.current_page = "search_results"
            st.session_state.selected_recipe_id = None # Clear selected ID
            st.rerun()
    else: # If no recipe ID selected, go back to search results
        st.session_state.current_page = "search_results"
        st.rerun()

# Default state if nothing is triggered yet
elif st.session_state.current_page == "search_input":
    st.info("Enter a recipe name above and click 'Search Recipes' to find recipes.")

StreamlitSecretNotFoundError: No secrets found. Valid paths for a secrets.toml file or secret directories are: /root/.streamlit/secrets.toml, /content/.streamlit/secrets.toml

In [ ]:
# Install required libraries
!pip install requests streamlit

In [ ]:
# Create the .streamlit directory
!mkdir -p .streamlit

# Create the secrets.toml file with your API key
# Make sure to replace "YOUR_ACTUAL_SPOONACULAR_API_KEY_HERE" with your key!
# The single quotes around the entire string for echo are important.
!echo 'SPOONACULAR_API_KEY = "33323d39b2d54c42ba326fc2ec35cf32"' > .streamlit/secrets.toml

# (Optional) Verify the content of the secrets.toml file
print("Content of .streamlit/secrets.toml:")
!cat .streamlit/secrets.toml

Content of .streamlit/secrets.toml:
SPOONACULAR_API_KEY = "33323d39b2d54c42ba326fc2ec35cf32"


In [ ]:
%%writefile recipe_app.py
import streamlit as st
import requests
import json

# --- Configuration ---
# Get API key from Streamlit secrets
API_KEY = st.secrets["SPOONACULAR_API_KEY"] # This is the correct way to access it
BASE_URL = "https://api.spoonacular.com/"

# --- Spoonacular API Functions ---

def search_recipes(query, number=10):
    """Searches for recipes by name."""
    endpoint = "recipes/complexSearch"
    params = {
        "query": query,
        "number": number,
        "apiKey": API_KEY
    }
    try:
        response = requests.get(f"{BASE_URL}{endpoint}", params=params)
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json().get("results", [])
    except requests.exceptions.RequestException as e:
        st.error(f"Error searching for recipes: {e}")
        return []

def get_recipe_details(recipe_id):
    """Fetches detailed information for a specific recipe."""
    endpoint = f"recipes/{recipe_id}/information"
    params = {
        "includeNutrition": True, # Get nutrition data for calories
        "apiKey": API_KEY
    }
    try:
        response = requests.get(f"{BASE_URL}{endpoint}", params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        st.error(f"Error fetching recipe details: {e}")
        return None

# --- Streamlit UI Layout ---

st.set_page_config(page_title="Python Recipe Finder", layout="centered")

st.title("🍜 Python Recipe Finder")

# Input for recipe name
recipe_name_input = st.text_input(
    "Enter a recipe name (e.g., pasta, chicken soup)",
    key="recipe_name_input"
)

# Search button
if st.button("Search Recipes"):
    if recipe_name_input:
        st.session_state.search_query = recipe_name_input # Store query in session state
        st.session_state.current_page = "search_results" # Navigate to search results
        st.rerun() # Rerun the app to show results
    else:
        st.warning("Please enter a recipe name to search.")

# --- Navigation and Display Logic ---

# Initialize session state for navigation if not already set
if 'current_page' not in st.session_state:
    st.session_state.current_page = "search_input" # Default page
if 'search_query' not in st.session_state:
    st.session_state.search_query = ""
if 'selected_recipe_id' not in st.session_state:
    st.session_state.selected_recipe_id = None


# Display logic based on current_page in session state
if st.session_state.current_page == "search_results":
    if st.session_state.search_query:
        st.subheader(f"Search Results for '{st.session_state.search_query}'")
        recipes = search_recipes(st.session_state.search_query)

        if recipes:
            for recipe in recipes:
                col1, col2 = st.columns([3, 1])
                with col1:
                    st.write(f"**{recipe['title']}**")
                with col2:
                    # Store selected recipe ID in session state and change page
                    if st.button("View Details", key=f"view_{recipe['id']}"):
                        st.session_state.selected_recipe_id = recipe['id']
                        st.session_state.current_page = "recipe_details"
                        st.rerun() # Rerun to show details
            st.markdown("---") # Separator
            if st.button("Back to Search Input", key="back_to_input_from_results"):
                st.session_state.current_page = "search_input"
                st.session_state.search_query = ""
                st.rerun()
        else:
            st.info("No recipes found for your search. Try a different name.")
            if st.button("Back to Search Input", key="back_to_input_no_results"):
                st.session_state.current_page = "search_input"
                st.session_state.search_query = ""
                st.rerun()
    else: # If search_query is empty, go back to input state
        st.session_state.current_page = "search_input"
        st.rerun()

elif st.session_state.current_page == "recipe_details":
    if st.session_state.selected_recipe_id:
        st.subheader("Recipe Details")
        details = get_recipe_details(st.session_state.selected_recipe_id)

        if details:
            st.markdown(f"## {details.get('title', 'N/A')}")
            st.write(f"**Preparation Time:** {details.get('readyInMinutes', 'N/A')} minutes")
            st.write(f"**Servings:** {details.get('servings', 'N/A')}")

            # Extract Calories
            calories = "N/A"
            if details.get('nutrition') and details['nutrition'].get('nutrients'):
                for nutrient in details['nutrition']['nutrients']:
                    if nutrient.get('title') == 'Calories':
                        calories = f"{nutrient.get('amount', 'N/A')} {nutrient.get('unit', '')}"
                        break
            st.write(f"**Calories:** {calories}")

            if details.get('sourceUrl'):
                st.markdown(f"**Source:** [Link to Recipe]({details['sourceUrl']})")

            st.markdown("### Ingredients:")
            if details.get('extendedIngredients'):
                ingredients_list = "\n".join([f"- {ing['original']}" for ing in details['extendedIngredients']])
                st.markdown(ingredients_list)
            else:
                st.info("No detailed ingredients available.")

            st.markdown("### Instructions:")
            if details.get('analyzedInstructions'):
                # Handle structured steps
                for instruction_set in details['analyzedInstructions']:
                    steps_md = ""
                    for step in instruction_set.get('steps', []):
                        steps_md += f"{step.get('number')}. {step.get('step')}\n"
                    st.markdown(steps_md)
            elif details.get('instructions'):
                # Handle single string instructions (might be HTML)
                st.markdown(details['instructions'], unsafe_allow_html=True) # Use unsafe_allow_html if instructions can contain HTML
            else:
                st.info("No detailed instructions available.")
        else:
            st.warning("Could not load recipe details.")

        # Back button
        if st.button("Back to Search Results", key="back_to_results_from_details"):
            st.session_state.current_page = "search_results"
            st.session_state.selected_recipe_id = None # Clear selected ID
            st.rerun()
    else: # If no recipe ID selected, go back to search results
        st.session_state.current_page = "search_results"
        st.rerun()

# Default state if nothing is triggered yet
elif st.session_state.current_page == "search_input":
    st.info("Enter a recipe name above and click 'Search Recipes' to find recipes.")

Writing recipe_app.py


In [ ]:
# Run the Streamlit app
!streamlit run recipe_app.py &>/dev/null& npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧your url is: https://early-readers-beam.loca.lt
^C


In [ ]:
# Install required libraries
!pip install requests streamlit pyngrok

In [ ]:
# Authenticate ngrok with your authtoken
# IMPORTANT: Replace YOUR_NGROK_AUTHTOKEN_HERE with your actual ngrok authtoken
from pyngrok import ngrok
ngrok.set_auth_token("2xuatcbmHbDx33buojBGNGUkDGn_4ovYEiwmVDLXDwBC4eFFc")

In [ ]:
# Create the .streamlit directory
!mkdir -p .streamlit

# Create the secrets.toml file with your Spoonacular API key
# IMPORTANT: Replace "YOUR_ACTUAL_SPOONACULAR_API_KEY_HERE" with your Spoonacular key!
!echo 'SPOONACULAR_API_KEY = "33323d39b2d54c42ba326fc2ec35cf32"' > .streamlit/secrets.toml

# (Optional) Verify the content of the secrets.toml file
print("Content of .streamlit/secrets.toml:")
!cat .streamlit/secrets.toml

Content of .streamlit/secrets.toml:
SPOONACULAR_API_KEY = "33323d39b2d54c42ba326fc2ec35cf32"


In [ ]:
%%writefile recipe_app.py
import streamlit as st
import requests
import json

# --- Configuration ---
# Get API key from Streamlit secrets
API_KEY = st.secrets["SPOONACULAR_API_KEY"] # This is the correct way to access it
BASE_URL = "https://api.spoonacular.com/"

# --- Spoonacular API Functions ---

def search_recipes(query, number=10):
    """Searches for recipes by name."""
    endpoint = "recipes/complexSearch"
    params = {
        "query": query,
        "number": number,
        "apiKey": API_KEY
    }
    try:
        response = requests.get(f"{BASE_URL}{endpoint}", params=params)
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json().get("results", [])
    except requests.exceptions.RequestException as e:
        st.error(f"Error searching for recipes: {e}")
        return []

def get_recipe_details(recipe_id):
    """Fetches detailed information for a specific recipe."""
    endpoint = f"recipes/{recipe_id}/information"
    params = {
        "includeNutrition": True, # Get nutrition data for calories
        "apiKey": API_KEY
    }
    try:
        response = requests.get(f"{BASE_URL}{endpoint}", params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        st.error(f"Error fetching recipe details: {e}")
        return None

# --- Streamlit UI Layout ---

st.set_page_config(page_title="Python Recipe Finder", layout="centered")

st.title("🍜 Python Recipe Finder")

# Input for recipe name
recipe_name_input = st.text_input(
    "Enter a recipe name (e.g., pasta, chicken soup)",
    key="recipe_name_input"
)

# Search button
if st.button("Search Recipes"):
    if recipe_name_input:
        st.session_state.search_query = recipe_name_input # Store query in session state
        st.session_state.current_page = "search_results" # Navigate to search results
        st.rerun() # Rerun the app to show results
    else:
        st.warning("Please enter a recipe name to search.")

# --- Navigation and Display Logic ---

# Initialize session state for navigation if not already set
if 'current_page' not in st.session_state:
    st.session_state.current_page = "search_input" # Default page
if 'search_query' not in st.session_state:
    st.session_state.search_query = ""
if 'selected_recipe_id' not in st.session_state:
    st.session_state.selected_recipe_id = None


# Display logic based on current_page in session state
if st.session_state.current_page == "search_results":
    if st.session_state.search_query:
        st.subheader(f"Search Results for '{st.session_state.search_query}'")
        recipes = search_recipes(st.session_state.search_query)

        if recipes:
            for recipe in recipes:
                col1, col2 = st.columns([3, 1])
                with col1:
                    st.write(f"**{recipe['title']}**")
                with col2:
                    # Store selected recipe ID in session state and change page
                    if st.button("View Details", key=f"view_{recipe['id']}"):
                        st.session_state.selected_recipe_id = recipe['id']
                        st.session_state.current_page = "recipe_details"
                        st.rerun() # Rerun to show details
            st.markdown("---") # Separator
            if st.button("Back to Search Input", key="back_to_input_from_results"):
                st.session_state.current_page = "search_input"
                st.session_state.search_query = ""
                st.rerun()
        else:
            st.info("No recipes found for your search. Try a different name.")
            if st.button("Back to Search Input", key="back_to_input_no_results"):
                st.session_state.current_page = "search_input"
                st.session_state.search_query = ""
                st.rerun()
    else: # If search_query is empty, go back to input state
        st.session_state.current_page = "search_input"
        st.rerun()

elif st.session_state.current_page == "recipe_details":
    if st.session_state.selected_recipe_id:
        st.subheader("Recipe Details")
        details = get_recipe_details(st.session_state.selected_recipe_id)

        if details:
            st.markdown(f"## {details.get('title', 'N/A')}")
            st.write(f"**Preparation Time:** {details.get('readyInMinutes', 'N/A')} minutes")
            st.write(f"**Servings:** {details.get('servings', 'N/A')}")

            # Extract Calories
            calories = "N/A"
            if details.get('nutrition') and details['nutrition'].get('nutrients'):
                for nutrient in details['nutrition']['nutrients']:
                    if nutrient.get('title') == 'Calories':
                        calories = f"{nutrient.get('amount', 'N/A')} {nutrient.get('unit', '')}"
                        break
            st.write(f"**Calories:** {calories}")

            if details.get('sourceUrl'):
                st.markdown(f"**Source:** [Link to Recipe]({details['sourceUrl']})")

            st.markdown("### Ingredients:")
            if details.get('extendedIngredients'):
                ingredients_list = "\n".join([f"- {ing['original']}" for ing in details['extendedIngredients']])
                st.markdown(ingredients_list)
            else:
                st.info("No detailed ingredients available.")

            st.markdown("### Instructions:")
            if details.get('analyzedInstructions'):
                # Handle structured steps
                for instruction_set in details['analyzedInstructions']:
                    steps_md = ""
                    for step in instruction_set.get('steps', []):
                        steps_md += f"{step.get('number')}. {step.get('step')}\n"
                    st.markdown(steps_md)
            elif details.get('instructions'):
                # Handle single string instructions (might be HTML)
                st.markdown(details['instructions'], unsafe_allow_html=True) # Use unsafe_allow_html if instructions can contain HTML
            else:
                st.info("No detailed instructions available.")
        else:
            st.warning("Could not load recipe details.")

        # Back button
        if st.button("Back to Search Results", key="back_to_results_from_details"):
            st.session_state.current_page = "search_results"
            st.session_state.selected_recipe_id = None # Clear selected ID
            st.rerun()
    else: # If no recipe ID selected, go back to search results
        st.session_state.current_page = "search_results"
        st.rerun()

# Default state if nothing is triggered yet
elif st.session_state.current_page == "search_input":
    st.info("Enter a recipe name above and click 'Search Recipes' to find recipes.")

Overwriting recipe_app.py


In [ ]:
# Run the Streamlit app in the background and create an ngrok tunnel
import subprocess
import time
from pyngrok import ngrok

# Start Streamlit in the background
# We pipe output to null to avoid cluttering the Colab output
process = subprocess.Popen(["streamlit", "run", "recipe_app.py"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Give Streamlit a moment to start up
time.sleep(5)

# Open an ngrok tunnel to the Streamlit app's port (8501)
# The ngrok.connect() call returns the public URL
public_url = ngrok.connect(8501)

print(f"Your Streamlit app is accessible at: {public_url}")
print("Please copy and paste this URL into your browser.")
print("\nTo stop the app and ngrok tunnel, run ngrok.kill() in a new cell.")

Your Streamlit app is accessible at: NgrokTunnel: "https://f976-34-105-72-210.ngrok-free.app" -> "http://localhost:8501"
Please copy and paste this URL into your browser.

To stop the app and ngrok tunnel, run ngrok.kill() in a new cell.


it is working fine